### Code Implementation

In [1]:
# Importing necessary libraries
import keras
from keras.datasets import mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pickle

### Loading Dataset

In [7]:
# # Loading the dataset and perform splitting
# data_list = []
# labels_list = []
# classes_list = 43

# imgs_path = "../input/gtsrb-german-traffic-sign/Train"
# for i in range(classes_list):
#     i_path = os.path.join(imgs_path, str(i)) #0-42
#     for img in os.listdir(i_path):
#         im = Image.open(i_path +'/'+ img)
#         im = im.resize((32,32))
#         im = np.array(im)
#         data_list.append(im)
#         labels_list.append(i)
# data = np.array(data_list)
# labels = np.array(labels_list)

In [2]:
# Opening file for reading in binary mode
with open('D:\Project\Traffic Sign Detection\DATABASE/data8.pickle', 'rb') as f:
    gray_data = pickle.load(f, encoding='latin1')  # dictionary type
    
    
# Making channels come at the end
gray_data['x_train'] = gray_data['x_train'].transpose(0, 2, 3, 1)
gray_data['x_validation'] = gray_data['x_validation'].transpose(0, 2, 3, 1)
gray_data['x_test'] = gray_data['x_test'].transpose(0, 2, 3, 1)

gray_train_imgs = gray_data['x_train']
gray_test_imgs = gray_data['x_test']
gray_train_labels = gray_data['y_train']
gray_test_labels = gray_data['y_test']

In [3]:
gray_train_labels[gray_train_labels <= 10] = 0 #cycle
gray_train_labels[gray_train_labels >= 32] = 0 #cycle
gray_train_labels[(gray_train_labels == 15) |(gray_train_labels == 16)| (gray_train_labels ==17)] = 0 #cycle
gray_train_labels[(gray_train_labels >= 18) & (gray_train_labels <= 31)] = 1 #triangle
gray_train_labels[(gray_train_labels == 13) | (gray_train_labels == 11)] = 1 #triangle
gray_train_labels[gray_train_labels == 14] = 2 # 6
gray_train_labels[gray_train_labels == 12] = 3 # 4


gray_test_labels[gray_test_labels <= 10] = 0 #cycle
gray_test_labels[gray_test_labels >= 32] = 0 #cycle
gray_test_labels[(gray_test_labels == 15) |(gray_test_labels == 16)| (gray_test_labels ==17)] = 0 #cycle
gray_test_labels[(gray_test_labels >= 18) & (gray_test_labels <= 31)] = 1 #triangle
gray_test_labels[(gray_test_labels == 13) | (gray_test_labels == 11)] = 1 #triangle
gray_test_labels[gray_test_labels == 14] = 2 # 6
gray_test_labels[gray_test_labels == 12] = 3 # 4

### Image Data Preprocessing

In [4]:
# Peforming reshaping operation
# x_train = gray_train_imgs.reshape(gray_train_imgs.shape[0], 28, 28, 1)
# x_test = gray_test_imgs.reshape(gray_test_imgs.shape[0], 28, 28, 1)

x_train = gray_train_imgs
x_test = gray_test_imgs

# # Normalization
# x_train = x_train / 255
# x_test = x_test / 255

num_class_shape = 4

# One Hot Encoding
y_train = keras.utils.to_categorical(gray_train_labels, num_class_shape)
y_test = keras.utils.to_categorical(gray_test_labels, num_class_shape)

In [5]:
a, x_train, b, y_train = train_test_split(gray_train_imgs, y_train, test_size=0.36, random_state=42)

### LeNet Model Architecture

In [5]:
keras.backend.clear_session()

In [6]:
# Building the Model Architecture
model = Sequential()
# Select 6 feature convolution kernels with a size of 5 * 5 (without offset), and get 66 feature maps. The size of each feature map is 32−5 + 1 = 2832−5 + 1 = 28.
# That is, the number of neurons has been reduced from 10241024 to 28 ∗ 28 = 784 28 ∗ 28 = 784.
# Parameters between input layer and C1 layer: 6 ∗ (5 ∗ 5 + 1)
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 1)))
# The input of this layer is the output of the first layer, which is a 28 * 28 * 6 node matrix.
# The size of the filter used in this layer is 2 * 2, and the step length and width are both 2, so the output matrix size of this layer is 14 * 14 * 6.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 14 * 14 * 6, the filter size used is 5 * 5, and the depth is 16. This layer does not use all 0 padding, and the step size is 1.
# The output matrix size of this layer is 10 * 10 * 16. This layer has 5 * 5 * 6 * 16 + 16 = 2416 parameters
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu'))
# The input matrix size of this layer is 10 * 10 * 16. The size of the filter used in this layer is 2 * 2, and the length and width steps are both 2, so the output matrix size of this layer is 5 * 5 * 16.
model.add(MaxPooling2D(pool_size=(2, 2)))
# The input matrix size of this layer is 5 * 5 * 16. This layer is called a convolution layer in the LeNet-5 paper, but because the size of the filter is 5 * 5, #
# So it is not different from the fully connected layer. If the nodes in the 5 * 5 * 16 matrix are pulled into a vector, then this layer is the same as the fully connected layer.
# The number of output nodes in this layer is 120, with a total of 5 * 5 * 16 * 120 + 120 = 48120 parameters.
model.add(Flatten())
model.add(Dense(120, activation='relu'))
# The number of input nodes in this layer is 120 and the number of output nodes is 84. The total parameter is 120 * 84 + 84 = 10164 (w + b)
model.add(Dense(84, activation='relu'))
# The number of input nodes in this layer is 84 and the number of output nodes is 10. The total parameter is 84 * 10 + 10 = 850
model.add(Dense(4, activation='softmax'))

In [7]:
model.compile(loss=keras.metrics.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
680/680 [==============================] - 9s 8ms/step - loss: 0.0870 - accuracy: 0.9721
Epoch 2/20
680/680 [==============================] - 5s 8ms/step - loss: 0.0134 - accuracy: 0.9962
Epoch 3/20
680/680 [==============================] - 5s 8ms/step - loss: 0.0075 - accuracy: 0.9977
Epoch 4/20
680/680 [==============================] - 5s 8ms/step - loss: 0.0048 - accuracy: 0.9984
Epoch 5/20
680/680 [==============================] - 6s 8ms/step - loss: 0.0034 - accuracy: 0.9990
Epoch 6/20
680/680 [==============================] - 6s 8ms/step - loss: 0.0031 - accuracy: 0.9990
Epoch 7/20
680/680 [==============================] - 6s 9ms/step - loss: 0.0027 - accuracy: 0.9990
Epoch 8/20
680/680 [==============================] - 6s 8ms/step - loss: 0.0020 - accuracy: 0.9994
Epoch 9/20
680/680 [==============================] - 6s 9ms/step - loss: 0.0042 - accuracy: 0.9988
Epoch 10/20
680/680 [==============================] - 6s 8ms/step - loss: 7.9519e-04 - accuracy: 0.

### data4

In [23]:
score = model.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

395/395 [==============================] - 3s 8ms/step - loss: 0.0302 - accuracy: 0.9952
Test Loss: 0.030224520713090897
Test accuracy: 0.9951702356338501


In [24]:
pred = model.predict(x_test).argmax(axis = 1)
cr = classification_report(y_test.argmax(axis = -1) , pred , output_dict = True)
print(cr)

395/395 [==============================] - 2s 4ms/step
{'0': {'precision': 0.9966903652856092, 'recall': 0.9964460784313726, 'f1-score': 0.9965682068880991, 'support': 8160}, '1': {'precision': 0.9929098128190584, 'recall': 0.9974358974358974, 'f1-score': 0.995167708925526, 'support': 3510}, '2': {'precision': 0.9961685823754789, 'recall': 0.9629629629629629, 'f1-score': 0.9792843691148776, 'support': 270}, '3': {'precision': 0.9883211678832117, 'recall': 0.981159420289855, 'f1-score': 0.9847272727272727, 'support': 690}, 'accuracy': 0.9951702296120348, 'macro avg': {'precision': 0.9935224820908396, 'recall': 0.9845010897800219, 'f1-score': 0.9889368894139439, 'support': 12630}, 'weighted avg': {'precision': 0.9951713338722297, 'recall': 0.9951702296120348, 'f1-score': 0.9951626147567949, 'support': 12630}}


### data8 

In [9]:
score = model.evaluate(x_test, y_test)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

395/395 [==============================] - 2s 5ms/step - loss: 0.0222 - accuracy: 0.9968
Test Loss: 0.022216537967324257
Test accuracy: 0.9968329668045044


In [10]:
pred = model.predict(x_test).argmax(axis = 1)
cr = classification_report(y_test.argmax(axis = -1) , pred , output_dict = True)

395/395 [==============================] - 2s 4ms/step


In [11]:
print(cr)

{'0': {'precision': 0.9970631424375918, 'recall': 0.9985294117647059, 'f1-score': 0.9977957384276267, 'support': 8160}, '1': {'precision': 0.9965831435079726, 'recall': 0.9971509971509972, 'f1-score': 0.9968669894616919, 'support': 3510}, '2': {'precision': 0.9962962962962963, 'recall': 0.9962962962962963, 'f1-score': 0.9962962962962963, 'support': 270}, '3': {'precision': 0.9955621301775148, 'recall': 0.9753623188405797, 'f1-score': 0.9853587115666179, 'support': 690}, 'accuracy': 0.9968329374505146, 'macro avg': {'precision': 0.9963761781048439, 'recall': 0.9918347560131449, 'f1-score': 0.9940794339380582, 'support': 12630}, 'weighted avg': {'precision': 0.9968313496299459, 'recall': 0.9968329374505146, 'f1-score': 0.9968261179383167, 'support': 12630}}


In [12]:
model_save_dir = "D:\\Project\\Traffic Sign Detection\\TrainedMOdel\\final\\shape_classifier_model_data0_acc99.h5"  

model.save(model_save_dir)

In [ ]:
import joblib 

# save model with joblib 
filename = 'joblib_model.sav'
joblib.dump(model, filename)

In [ ]:
x_test[1].shape

## Hope you like it!

In [ ]:
from matplotlib import pyplot as plt
image = gray_test_imgs[5]#.transpose(2 , 0 ,1)
# plt.imshow(image, cmap=plt.get_cmap('gray'))
# plt.show()
y = model.predict(image)
print(y)